In [0]:
sampleData = [
('Tom Prescott', 'Furniture', 'Chairs', 4000),
('Quincy Jones', 'Furniture', 'Bookcases', 4000),
('Joseph Holt', 'Furniture', 'Tables', 5000),
('Alejandro Grove', 'Furniture', 'Furnishings', 8000),
('Adrian Barton', 'Office Supplies', 'Binders', 3000),
('Ken Lonsdale', 'Office Supplies', 'Supplies', 9000),
('Greg Guthrie', 'Office Supplies', 'Fasteners', 3000),
('Yoseph Carroll', 'Office Supplies', 'Storage', 3000),
('Sean Miller', 'Technology', 'Machines', 22000),
('Tamara Chand', 'Technology', 'Copiers', 18000),
('John Murray', 'Technology', 'Phones', 5000),
('Kelly Collister', 'Technology', 'Accessories', 3000)
]

# column names for dataframe
columns = ["Customer Name", "Category",
           "Sub Category", "Sales"]
 
# creating the dataframe df
df = spark.createDataFrame(data=sampleData,
                           schema=columns)

df.show()

+---------------+---------------+------------+-----+
|  Customer Name|       Category|Sub Category|Sales|
+---------------+---------------+------------+-----+
|   Tom Prescott|      Furniture|      Chairs| 4000|
|   Quincy Jones|      Furniture|   Bookcases| 4000|
|    Joseph Holt|      Furniture|      Tables| 5000|
|Alejandro Grove|      Furniture| Furnishings| 8000|
|  Adrian Barton|Office Supplies|     Binders| 3000|
|   Ken Lonsdale|Office Supplies|    Supplies| 9000|
|   Greg Guthrie|Office Supplies|   Fasteners| 3000|
| Yoseph Carroll|Office Supplies|     Storage| 3000|
|    Sean Miller|     Technology|    Machines|22000|
|   Tamara Chand|     Technology|     Copiers|18000|
|    John Murray|     Technology|      Phones| 5000|
|Kelly Collister|     Technology| Accessories| 3000|
+---------------+---------------+------------+-----+



In [0]:
from pyspark.sql.window import * # this is needed for window functions
from pyspark.sql.functions import * # this is needed for window functions
from pyspark.sql.types import  *  # thi shas to be used when you are trying to assign datatypes and struct fields and struct types
from pyspark.sql.column import * # this has to beused when you are trying to use the column functio in datafram

 to create a windo function in pyspark. there are 2 steps
step-1: assign a partition and order level, either table or specific column partiton level
step-2: write any function based on above partitiuon created

for SQL table level : over(order by sales desc)
pyspark code for table level: window.orderby(col("sales").desc())

for SQL partition level: over(partiton by category order by sales desc )
pyspark code for partition level: windows.partitionBy("category).orderBy(col("sales).desc())

In [0]:
#create a row_number() for above table based on sales descending

#step1: creatre a window partiton at table level
from pyspark.sql.window import Window
#step1: create a window partition at table level
from pyspark.sql.window import Window

# Corrected line:
table_partition = Window.orderBy(col("Sales").desc())

#step2: Create a row_number functgion
df_row = df.withColumn("row_number", row_number().over(table_partition))
df_row.show()

/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+---------------+---------------+------------+-----+----------+
|  Customer Name|       Category|Sub Category|Sales|row_number|
+---------------+---------------+------------+-----+----------+
|    Sean Miller|     Technology|    Machines|22000|         1|
|   Tamara Chand|     Technology|     Copiers|18000|         2|
|   Ken Lonsdale|Office Supplies|    Supplies| 9000|         3|
|Alejandro Grove|      Furniture| Furnishings| 8000|         4|
|    Joseph Holt|      Furniture|      Tables| 5000|         5|
|    John Murray|     Technology|      Phones| 5000|         6|
|   Tom Prescott|      Furniture|      Chairs| 4000|         7|
|   Quincy Jones|      Furniture|   Bookcases| 4000|         8|
|  Adrian Barton|Office Supplies|     Binders| 3000|         9|
|   Greg Guthrie|Office Supplies|   Fasteners| 3000|        10|
| Yoseph Carroll|Office Supplies|     Storage| 3000|        11|
|Kelly Collister|     Technology| Accessories| 3000|        12|
+---------------+---------------+-------

In [0]:
# create a row_number() for abov table based on category partition and sales descending

# step-1: Create a window partion at tabel level
window_cat = Window.partitionBy("Category").orderBy(col("Sales").desc())

#step-2: create a row_number function
df2 = df.withColumn("row_number", row_number().over(window_cat))\
         .withColumn("Rank",rank().over(window_cat))\
         .withColumn("dense_rank",dense_rank().over(window_cat))  
df2.show()

+---------------+---------------+------------+-----+----------+----+----------+
|  Customer Name|       Category|Sub Category|Sales|row_number|Rank|dense_rank|
+---------------+---------------+------------+-----+----------+----+----------+
|Alejandro Grove|      Furniture| Furnishings| 8000|         1|   1|         1|
|    Joseph Holt|      Furniture|      Tables| 5000|         2|   2|         2|
|   Tom Prescott|      Furniture|      Chairs| 4000|         3|   3|         3|
|   Quincy Jones|      Furniture|   Bookcases| 4000|         4|   3|         3|
|   Ken Lonsdale|Office Supplies|    Supplies| 9000|         1|   1|         1|
|  Adrian Barton|Office Supplies|     Binders| 3000|         2|   2|         2|
|   Greg Guthrie|Office Supplies|   Fasteners| 3000|         3|   2|         2|
| Yoseph Carroll|Office Supplies|     Storage| 3000|         4|   2|         2|
|    Sean Miller|     Technology|    Machines|22000|         1|   1|         1|
|   Tamara Chand|     Technology|     Co

In [0]:
# LEAD : get the next row value into current row
# syntax: lead("Colnmae",<no_of_nextcolumn>).over(<partition level>)

#Lag : get the last row value into current row
# syntax: lag("Colnmae",<no_of_previouscolumn>).over(<partition level>)

#get the previous row sales into current row

#step1 : create a window partition at table level

table_partition = Window.orderBy(col("Sales").desc())

# step2: Create a laf function

df_lag = df.withColumn("lag1",lag("Sales",1).over(table_partition))\
           .withColumn("lag2",lag("Sales",2).over(table_partition))
           
df_lag.show()

/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+---------------+---------------+------------+-----+-----+-----+
|  Customer Name|       Category|Sub Category|Sales| lag1| lag2|
+---------------+---------------+------------+-----+-----+-----+
|    Sean Miller|     Technology|    Machines|22000| NULL| NULL|
|   Tamara Chand|     Technology|     Copiers|18000|22000| NULL|
|   Ken Lonsdale|Office Supplies|    Supplies| 9000|18000|22000|
|Alejandro Grove|      Furniture| Furnishings| 8000| 9000|18000|
|    Joseph Holt|      Furniture|      Tables| 5000| 8000| 9000|
|    John Murray|     Technology|      Phones| 5000| 5000| 8000|
|   Tom Prescott|      Furniture|      Chairs| 4000| 5000| 5000|
|   Quincy Jones|      Furniture|   Bookcases| 4000| 4000| 5000|
|  Adrian Barton|Office Supplies|     Binders| 3000| 4000| 4000|
|   Greg Guthrie|Office Supplies|   Fasteners| 3000| 3000| 4000|
| Yoseph Carroll|Office Supplies|     Storage| 3000| 3000| 3000|
|Kelly Collister|     Technology| Accessories| 3000| 3000| 3000|
+---------------+--------

In [0]:
# Get the Next row sales into current row

# Step1 : create a window partition at table level
table_partition = Window.orderBy(col("Sales").desc())

#step2: Create a lead function
df_lead = df.withColumn("lead1",lead("Sales",1).over(table_partition))\
            .withColumn("lead2",lead("Sales",2).over(table_partition))
df_lead.show()

/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+---------------+---------------+------------+-----+-----+-----+
|  Customer Name|       Category|Sub Category|Sales|lead1|lead2|
+---------------+---------------+------------+-----+-----+-----+
|    Sean Miller|     Technology|    Machines|22000|18000| 9000|
|   Tamara Chand|     Technology|     Copiers|18000| 9000| 8000|
|   Ken Lonsdale|Office Supplies|    Supplies| 9000| 8000| 5000|
|Alejandro Grove|      Furniture| Furnishings| 8000| 5000| 5000|
|    Joseph Holt|      Furniture|      Tables| 5000| 5000| 4000|
|    John Murray|     Technology|      Phones| 5000| 4000| 4000|
|   Tom Prescott|      Furniture|      Chairs| 4000| 4000| 3000|
|   Quincy Jones|      Furniture|   Bookcases| 4000| 3000| 3000|
|  Adrian Barton|Office Supplies|     Binders| 3000| 3000| 3000|
|   Greg Guthrie|Office Supplies|   Fasteners| 3000| 3000| 3000|
| Yoseph Carroll|Office Supplies|     Storage| 3000| 3000| NULL|
|Kelly Collister|     Technology| Accessories| 3000| NULL| NULL|
+---------------+--------

In [0]:
# GET Top 2 sales from each category

partition_cat = Window.partitionBy("Category").orderBy(col("Sales").desc())

df_top2 = df.withColumn("row_number", row_number().over(partition_cat))\
            .filter(col("row_number") <= 2)
df_top2.show()

+---------------+---------------+------------+-----+----------+
|  Customer Name|       Category|Sub Category|Sales|row_number|
+---------------+---------------+------------+-----+----------+
|Alejandro Grove|      Furniture| Furnishings| 8000|         1|
|    Joseph Holt|      Furniture|      Tables| 5000|         2|
|   Ken Lonsdale|Office Supplies|    Supplies| 9000|         1|
|  Adrian Barton|Office Supplies|     Binders| 3000|         2|
|    Sean Miller|     Technology|    Machines|22000|         1|
|   Tamara Chand|     Technology|     Copiers|18000|         2|
+---------------+---------------+------------+-----+----------+



In [0]:
partition_cat = Window.partitionBy("Category").orderBy(col("Sales").desc())

df_top2 = df.withColumn("row_number", row_number().over(partition_cat))
df_top2 = df_top2.filter("row_number <= 2")
df_top2.show()

+---------------+---------------+------------+-----+----------+
|  Customer Name|       Category|Sub Category|Sales|row_number|
+---------------+---------------+------------+-----+----------+
|Alejandro Grove|      Furniture| Furnishings| 8000|         1|
|    Joseph Holt|      Furniture|      Tables| 5000|         2|
|   Ken Lonsdale|Office Supplies|    Supplies| 9000|         1|
|  Adrian Barton|Office Supplies|     Binders| 3000|         2|
|    Sean Miller|     Technology|    Machines|22000|         1|
|   Tamara Chand|     Technology|     Copiers|18000|         2|
+---------------+---------------+------------+-----+----------+



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# inirialize Spark session
spark = SparkSession.builder.appName("RemoveDuplicates").getOrCreate()

# sample data
data = [
    (1, 'Alice', '2024-01-01', 'A'), (1, 'Alice', '2024-01-02', 'B'), (1, 'Alice', '2024-01-03', 'C'), (1, 'Alice', '2024-01-04', 'D'),
    (2, 'Bob', '2024-02-01', 'E'), (2, 'Bob', '2024-02-02', 'F'), (2, 'Bob', '2024-02-03', 'G'), (2, 'Bob', '2024-02-04', 'H'),
    (3, 'Charlie', '2024-03-01', 'I'), (3, 'Charlie', '2024-03-02', 'J'), (3, 'Charlie', '2024-03-03', 'K'), (3, 'Charlie', '2024-03-04', 'L'),
    (4, 'David', '2024-04-01', 'M'), (4, 'David', '2024-04-02', 'N'), (4, 'David', '2024-04-03', 'O'), (4, 'David', '2024-04-04', 'P'),
    (5, 'Emma', '2024-05-01', 'Q'), (5, 'Emma', '2024-05-02', 'R'), (5, 'Emma', '2024-05-03', 'S'), (5, 'Emma', '2024-05-04', 'T')

]

# create Dataframe

df = spark.createDataFrame(data,["CustermID","CustomerName","Date","Category"])
df.show()


+---------+------------+----------+--------+
|CustermID|CustomerName|      Date|Category|
+---------+------------+----------+--------+
|        1|       Alice|2024-01-01|       A|
|        1|       Alice|2024-01-02|       B|
|        1|       Alice|2024-01-03|       C|
|        1|       Alice|2024-01-04|       D|
|        2|         Bob|2024-02-01|       E|
|        2|         Bob|2024-02-02|       F|
|        2|         Bob|2024-02-03|       G|
|        2|         Bob|2024-02-04|       H|
|        3|     Charlie|2024-03-01|       I|
|        3|     Charlie|2024-03-02|       J|
|        3|     Charlie|2024-03-03|       K|
|        3|     Charlie|2024-03-04|       L|
|        4|       David|2024-04-01|       M|
|        4|       David|2024-04-02|       N|
|        4|       David|2024-04-03|       O|
|        4|       David|2024-04-04|       P|
|        5|        Emma|2024-05-01|       Q|
|        5|        Emma|2024-05-02|       R|
|        5|        Emma|2024-05-03|       S|
|        5

In [0]:
#get latest data transaction for each customer

#step1: create a window partition at table level
table_partition = Window.partitionBy("CustomerName").orderBy(col("Date").desc())

df_ld = df.withColumn("row_number", row_number().over(table_partition))\
          .filter(col("row_number") == 1)
df_ld.show()

+---------+------------+----------+--------+----------+
|CustermID|CustomerName|      Date|Category|row_number|
+---------+------------+----------+--------+----------+
|        1|       Alice|2024-01-04|       D|         1|
|        2|         Bob|2024-02-04|       H|         1|
|        3|     Charlie|2024-03-04|       L|         1|
|        4|       David|2024-04-04|       P|         1|
|        5|        Emma|2024-05-04|       T|         1|
+---------+------------+----------+--------+----------+



In [0]:
df_ld = df.withColumn("row_n", row_number().over(table_partition))
df_ld = df_ld.filter("row_n =1")
df_ld.show()

+---------+------------+----------+--------+-----+
|CustermID|CustomerName|      Date|Category|row_n|
+---------+------------+----------+--------+-----+
|        1|       Alice|2024-01-04|       D|    1|
|        2|         Bob|2024-02-04|       H|    1|
|        3|     Charlie|2024-03-04|       L|    1|
|        4|       David|2024-04-04|       P|    1|
|        5|        Emma|2024-05-04|       T|    1|
+---------+------------+----------+--------+-----+



In [0]:
#initialize Spark session
spark = SparkSession.builder.appName("CreateDataFrame").getOrCreate()

# sample list
list1 = [
    (10000, 'John', '2025-01-01', 'A'),
    (10000, 'Jane', '2025-01-02', 'B'),
    (5000, 'Alice', '2025-01-03', 'C'),
    (5000, 'Bob', '2025-01-04', 'D'),
    (5000, 'Charlie', '2025-01-05', 'E'),
    (10000, 'David', '2025-01-06', 'F'),
    (3000, 'Emma', '2025-01-07', 'G'),
    (3000, 'Olivia', '2025-01-08', 'H'),
    (3000, 'Sophia', '2025-01-09', 'I')
]

# create dataframe
df = spark.createDataFrame(list1, ["CustermID", "CustomerName", "Date", "Category"])
df = df.select(df.CustermID.alias("SalesAmt"))
df.show()

+--------+
|SalesAmt|
+--------+
|   10000|
|   10000|
|    5000|
|    5000|
|    5000|
|   10000|
|    3000|
|    3000|
|    3000|
+--------+



In [0]:
part_rank = Window.orderBy(col("SalesAmt").desc())
df_rank = df.withColumn("RN", row_number().over(part_rank)).withColumn("Rank", rank().over(part_rank)).withColumn("dense_rank", dense_rank().over(part_rank))
df_rank.show()


/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+--------+---+----+----------+
|SalesAmt| RN|Rank|dense_rank|
+--------+---+----+----------+
|   10000|  1|   1|         1|
|   10000|  2|   1|         1|
|   10000|  3|   1|         1|
|    5000|  4|   4|         2|
|    5000|  5|   4|         2|
|    5000|  6|   4|         2|
|    3000|  7|   7|         3|
|    3000|  8|   7|         3|
|    3000|  9|   7|         3|
+--------+---+----+----------+

